<a href="https://colab.research.google.com/github/TanD18/Sentiment-Analysis-on-Zomato-Data/blob/main/Pre_Processing_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Library Import

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import re
import nltk
from itertools import groupby
from tqdm import tqdm,tqdm_notebook
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
import collections
import functools
import operator

In [ ]:
#For part of speech tagging
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import inflect
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.utils import lemmatize
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus.reader import wordnet
from bs4 import BeautifulSoup
import re, string, unicodedata

##Data Load

In [ ]:
!unzip /content/drive/MyDrive/archive.zip

Archive:  /content/drive/MyDrive/archive.zip
  inflating: zomato.csv              


In [ ]:
df=pd.read_csv('/content/zomato.csv')
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [ ]:
print('Restaurant Name:',df['name'][4])
df['reviews_list'][3]

Restaurant Name: Grand Village


'[(\'Rated 4.0\', "RATED\\n  Great food and proper Karnataka style full meals. Been there twice and was fully satisfied.. Will give 5 stars if it\'s well managed............"), (\'Rated 2.0\', \'RATED\\n  Reached the place at 3pm on Saturday. Half of the items on the menu were over. What was annoying was is the food was cold. The taste was also very average. Only dosa and holige were good. There were very few people in the restaurant and the service was still very slow. The waiters were all standing in one corner and talking. Had to call them repeatedly.\'), (\'Rated 4.0\', \'RATED\\n  Had been here, good food served and tasty,good place to go with freinds and family, should be first to get served well with all food. One of the good hotel there for this price.\'), (\'Rated 2.0\', \'RATED\\n  How can a Udupi restaurant be so dirty.the floor the walls the waiters are all stained in food.Why cant they clean it? The floor even had decorative colour paper every where and lot of food.Now com

##Road for Pre-processing:
1. Extract Ratings and Reviews separately
2. Removing html contents
<br> 

*   Remove html 
*   Remove url
*   Remove web associated

3. Remove Noise 
*   Round brackets
*   Punctuation
*   White Spaces

4. Normalization
*  contraction

5. Remove Low Frequency words
*  create dcitionary
*  find frequency of each word
*  find rare and working words
*  remove rare words from the review corpus



---



###Remove https

In [ ]:
def remove_slashes(content):
  return content.replace('\n',' ').replace('\\n',' ').replace('*\\',' ')

In [ ]:
def remove_url(data):
  return re.sub(r'https\S',' ',data)

In [ ]:
  removed_html = df['reviews_list'].apply(lambda x: remove_slashes(x))
  removed_web  = removed_html.apply(lambda x: remove_url(x))

###Segregate Ratings and Reviews

In [ ]:
#Separate out all the individual reviews of the restaurants
split_reviews=removed_web.apply(lambda x: x[9:].split(', (\'Rated '))

In [ ]:
split_reviews[4]

["4.0', 'RATED   Very good restaurant in neighbourhood. Buffet system is properly arranged. Variety of dishes along with garba dance and puppet show. Good spread of desserts. Live paratha / Kulcha making')",
 "4.0', 'RATED   Great service, overwhelming experience.  One of its kind experience. Grand village offers you some very desi food options.  Jaljeera, buttermilk, chat papdi, bhajiya, dosa, pav bhaji, noodles, and so many curries kulcha, roti, and garama garam Jalebies, gulab jamuns.  Beautiful ambience of village. People serving with smiles and with full heart !!  So here goes my overall ratings : Food : 4/5 Ambience : 3.5/5 Service : 4/5 Pocket friendly : 4/5')]"]

In [ ]:
#Removing Duplicates
split_reviews=split_reviews.apply(lambda x: list(set(x)))

In [ ]:
def extract_ratings(x):
  ratings=[]
  for i in x:
    ratings.append(i[:1])
  return ratings

def extract_reviews(x):
  reviews=[]
  for i in x:
    reviews.append(i[14:-2])
  return reviews

In [ ]:
#Split reviews and ratings
all_ratings=split_reviews.apply(lambda x: extract_ratings(x))
all_reviews=split_reviews.apply(lambda x: extract_reviews(x))  

In [ ]:
print("Restaurant name : ",df['name'][19])
print("Ratings:")
print(all_ratings[19])
print("Reviews:")
all_reviews[19]

Restaurant name :  360 Atoms Restaurant And Cafe
Ratings:
['5', '5', '4', '4']
Reviews:


[" Nice place to chill out and have a good time with friends. Nice ambience and good tasting food.  Good selection of foods for both veg and non veg.'",
 ' Friendly staffs , nice ambience colourful lights , hanging skeletons reasonable price , nice music gud for group of friends enjoyed a lot ??????',
 ' Hi... This is place to chill and experience different type of food. We recently visted and there was good treatment from staff n white sauce pasta , pizza , garlic bread stick, momos, was awesome. Great place.',
 ' Must experience..The service was impeccable.We were made to feel welcome and dishes very reasonably priced. Before we had even finished our plates we were planning the next time we would eat here.']

In [ ]:
#flatten out ratings and reviews
rating_list=[rating for restaurant_ratings in all_ratings for rating in restaurant_ratings]
review_list=[review for restaurant_reviews in all_reviews for review in restaurant_reviews]
len(rating_list)

558778

###Checking for Non-Numerical Rating Values

In [ ]:
non_numeric_rating=[i for i, j in enumerate(rating_list) if not j.isnumeric()]
print("Number of non numeric rating instances",len(non_numeric_rating))

Number of non numeric rating instances 7595


###Removing non numeric ratings along with their reviews

In [ ]:
#pop-ing in reverse order of the index
for i in non_numeric_rating[::-1]:
  print(i)
  rating_list.pop(i)
  review_list.pop(i)

Streaming output truncated to the last 5000 lines.
367020
366315
365834
365647
365147
365146
365144
365143
365142
365128
365116
365115
365113
365112
365111
365107
365053
365052
365051
365050
365032
365000
364999
364998
364997
364994
364991
364990
364989
364988
364966
364964
364887
364824
363473
363421
363244
363223
363222
363174
363173
363169
363163
363160
363146
363145
363122
363088
363079
363049
363048
363046
363000
362925
362924
362923
362922
362918
362917
362916
362915
362905
362904
362903
362902
362901
362900
362857
362856
362855
362854
362850
362842
362834
362833
362832
362831
362830
362815
362814
362813
362812
362811
362810
362809
362807
362806
362805
362804
362799
362798
362797
362796
362743
362741
362740
362707
362706
362653
362602
362601
362600
362599
362598
362285
362270
362269
362266
362265
362264
362263
362262
362261
362260
362259
362258
362257
362256
362255
362254
362252
362250
362247
362246
362243
362242
362240
362237
362234
362233
362218
362217
362216
362215
362214
3622

In [ ]:
[i for i, j in enumerate(rating_list) if not j.isnumeric()]

[]

###Convert rating value from String to Integer datatype

In [ ]:
rating_list=list(map(int,rating_list))
rating_list[123]

4

###Remove Noise

In [ ]:
#Remove punctuation
def remove_punc(contents):
  punc_list=(string.punctuation).replace
  table=contents.maketrans(string.punctuation,' ' * len(string.punctuation))
  return contents.translate(table)
  
#Remove white spaces
def remove_white_space(data):
  return ' '.join(data.split())

#Remove word containing non ascii characters
def remove_non_ascii_words(contents):
  string_ascii = ' '.join([ token for token in contents.split() if token.isascii() ])
  return string_ascii

#Noise Removal
def remove_noise(contents):
  remove_pun=remove_punc(contents)
  remove_spaces=remove_white_space(remove_pun)
  remove_non_ascii=remove_non_ascii_words(remove_spaces)
  return remove_non_ascii

In [ ]:
review_no_noise=[remove_noise(i) for i in review_list]
review_no_noise

['The best thing about the place is x92s ambiance Second best thing was yummy food We try buffet and buffet food was not disappointed us Test Quality Service Staff was very professional and friendly Overall experience was excellent subirmajumder85 wixsite com',
 'The place is nice and comfortable Food wise all jalea outlets maintain a good standard The soya chaap was a standout dish Clearly one of trademark dish as per me and a must try The only concern is the parking It very congested and limited to just 5cars The basement parking is very steep and makes it cumbersome',
 'The place is nice and comfortable Food wise all jalea outlets maintain a good standard The soya chaap was a standout dish Clearly one of trademark dish as per me and a must try The only concern is the parking It very congested and limited to just 5cars The basement parking is very steep and makes it cumbersome',
 'A beautiful place to dine in The interiors take you back to the Mughal era The lightings are just perfec

###Normalization

In [ ]:
#text to lower case
def to_lower(contents):
  return contents.lower()
#tokenize words
nltk.download('punkt')
def tokenize(contents):
  return nltk.word_tokenize(contents)
#remove_stopwords
nltk.download('stopwords')
def remove_stopwords(contents):
  cachedStopWords = stopwords.words("english")[:121]
  include_words=['until','above','below','down','up','under','over','out','why','how','all','any','both','each']
  for i in include_words:
    cachedStopWords.remove(i)
  no_stopwords=[]
  for i in tqdm(contents,desc='Stopword Removal'):
    no_stopwords.append([j for j in i if j not in cachedStopWords])
  return no_stopwords
tokens=[]
for i in review_no_noise:
  review_tolower=to_lower(i)
  tokens.append(nltk.word_tokenize(review_tolower))
no_stopwords=remove_stopwords(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Stopword Removal: 100%|██████████| 551183/551183 [00:55<00:00, 9934.06it/s] 


In [ ]:
with open('no_stopwords','wb') as fp:
  pickle.dump(no_stopwords,fp)

###Lemmatization

In [ ]:
#remove duplicate reviews 
arr=np.array(no_stopwords)
unique=list(np.unique(arr))

In [ ]:
rating_list_working=[]
for i in tqdm(unique,desc='Removing Ratings based on duplicate reviews'):
  index=no_stopwords.index(i)
  rating_list_working.append(rating_list[index])

Removing Ratings based on duplicate reviews: 100%|██████████| 134020/134020 [13:42<00:00, 162.89it/s]


In [ ]:
#POS tagging of the review words
pos_tag=[]
for i in tqdm(unique,desc='POS Tagging'):
  pos_tag.append(nltk.pos_tag(i))

POS Tagging: 100%|██████████| 134020/134020 [06:02<00:00, 369.66it/s]


In [ ]:
#Limiting the working POS word tags to Adjective, Verb, Noun, Adverb
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV

In [ ]:
wordnet_tagged=[]
for i in tqdm(pos_tag,desc='Tag conversion'):
  wordnet_tagged.append(list(map(lambda x: (x[0], pos_tagger(x[1])), i)))

Tag conversion: 100%|██████████| 134020/134020 [00:04<00:00, 30953.67it/s]


In [ ]:
def lemmatization(contents):
  lem = WordNetLemmatizer()
  lem_list=[]
  for i in tqdm(contents,desc='Lemmatization'):
    lem_review=[]
    for j in i:
      if j[1] is None:
        lem_review.append(j[0])
      else:
        lem_review.append(lem.lemmatize(j[0],j[1]))
    lem_list.append(lem_review)
  return lem_list

In [ ]:
lemmatized_review=lemmatization(wordnet_tagged)

Lemmatization: 100%|██████████| 134020/134020 [00:24<00:00, 5370.22it/s]


###  Remove rare words and Create Dictionary

In [ ]:
review_dct=Dictionary(lemmatized_review)

In [ ]:
corpus = [review_dct.doc2bow(sent) for sent in tqdm(lemmatized_review,desc='Term Frequency')]
vocab_tf_row=[dict(i) for i in corpus]
counter=collections.Counter()
for i in tqdm(vocab_tf_row,desc='Accumulating Frequencies'):
  counter.update(i)

Accumulating Frequencies: 100%|██████████| 134020/134020 [00:01<00:00, 99653.28it/s]


In [ ]:
#converting dictionary object from of counter
res=dict(counter)
res[5]

1631

In [ ]:
#creating list of word indexes in decreasing order of their frequency
sorted_res=sorted(res,key=res.get,reverse=True)
sorted_res

[92,
 98,
 55,
 10,
 301,
 32,
 167,
 26,
 119,
 16,
 9,
 151,
 72,
 61,
 115,
 73,
 130,
 38,
 450,
 122,
 59,
 192,
 186,
 66,
 253,
 6,
 202,
 382,
 13,
 235,
 638,
 118,
 79,
 236,
 444,
 120,
 24,
 133,
 246,
 568,
 67,
 339,
 408,
 390,
 162,
 1,
 90,
 84,
 376,
 735,
 69,
 96,
 331,
 562,
 570,
 254,
 386,
 137,
 94,
 252,
 239,
 425,
 705,
 764,
 1001,
 215,
 48,
 21,
 325,
 405,
 742,
 527,
 623,
 399,
 85,
 660,
 116,
 413,
 878,
 47,
 88,
 787,
 175,
 1509,
 18,
 983,
 306,
 618,
 54,
 809,
 437,
 381,
 195,
 87,
 616,
 187,
 647,
 217,
 15,
 161,
 782,
 409,
 315,
 226,
 238,
 103,
 91,
 947,
 8,
 612,
 123,
 467,
 410,
 429,
 209,
 33,
 838,
 496,
 155,
 671,
 74,
 245,
 272,
 109,
 841,
 240,
 698,
 279,
 266,
 19,
 49,
 287,
 502,
 468,
 668,
 1278,
 77,
 50,
 655,
 251,
 481,
 465,
 869,
 1032,
 627,
 1227,
 938,
 307,
 4,
 933,
 887,
 478,
 213,
 508,
 649,
 984,
 536,
 176,
 407,
 105,
 299,
 421,
 402,
 548,
 132,
 267,
 347,
 545,
 778,
 1414,
 282,
 442,
 78,
 108,

In [ ]:
for i in review_dct:
  if review_dct[i]=='first':
    print(i)

467


In [ ]:
#Creating list of rare words and index having frequency less than 5 in the corpus
#Creating list of rare words and index having frequency more than or equal to 5 in the corpus
working_words_ids=[]
rare_words_ids=[]
for i,j in tqdm(res.items(),desc='Counting low Frequent indices'):
  if j<5:
    rare_words_ids.append(i)
  else:
    working_words_ids.append(i)

working_words=[review_dct[i] for i in working_words_ids]
rare_words=[review_dct[i] for i in rare_words_ids]

len(working_words),len(rare_words)

Counting low Frequent indices: 100%|██████████| 54320/54320 [00:00<00:00, 1192033.70it/s]


(15163, 39157)

In [ ]:
#Flagging reviews with no working words 
eliminated_word=[]
check_list=[]
eliminate_index=[]
for index,review in enumerate(tqdm(lemmatized_review)):
  flag=0
  for words in review:
    if(words in working_words):
      flag=1
      check_list.append(1)
      break

  if (flag==0):
    check_list.append(0)
    eliminate_index.append(index)
    eliminated_word.append(review)

100%|██████████| 134020/134020 [00:06<00:00, 19974.33it/s]


In [ ]:
#Remove reviews with only rare words
for i in eliminate_index[::-1]:
  print(lemmatized_review[i])
  lemmatized_review.pop(i)
  rating_list_working.pop(i)

['zomto', 'supre', 'delieyre']
['ygvn']
['yas']
['yammiee']
['yamiiiiiiii']
['yaaaaakkk']
['verygood']
['verybad']
['unsatis']
['thuuu', 'daridra']
['tgood']
['tesry']
['tastyyyy']
['suprrrbbb']
['supr']
['suppee']
['suppar']
['superrrrrrrr']
['superrrr']
['superr']
['superbbbb']
['supebbbb']
['supbr']
['supar']
['succubus']
['sprb']
['spr', 'pa']
['spr']
['spicyyyy']
['spb']
['soso']
['satified']
['saficient']
['rrccg']
['ridden']
['rayautdo']
['ossam']
['osmm', 'nfj']
['osmee']
['osammm']
['okkk']
['okkii']
['notgood']
['notbad']
['noo']
['nm']
['nids']
['nid']
['niceeeeeee']
['niceee']
['nicee']
['nicd']
['ncy']
['mazo', 'ayo']
['mayosie']
['mayoneese']
['mastu']
['mastt']
['lovd']
['liquidity']
['lemone']
['jwjqj']
['ju']
['jsnznz']
['jahvs']
['itsworest']
['irsw']
['idnt']
['huguvihiihhftgdddfcvhjjjbhfiu']
['hud']
['hshehehe']
['hhhjjghjjj']
['hhhgzjgzigxkhfljckhfkhcohfihxihxjgohflhdkhxlhxkgxkgxkgxjgxgxkgxkgxjgxnc', 'ncj', 'jgxkhfohxohflhxigdlhohcgcjhfhcohchfjuufjfoufpjfpjgpugjflj

In [ ]:
#Creating a review corpus with working words having no rare words
working_corpus=[]
difference_list=[]
for i in tqdm(lemmatized_review):
  working_corpus.append([j for j in i if j in working_words])
len(working_corpus)

100%|██████████| 133896/133896 [04:01<00:00, 555.45it/s]


133896

In [ ]:
working_dictionary=Dictionary(working_corpus)